In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
import random

In [2]:
class Contraction:
    """
    equations describing muscle contraction dynamics, length and forces
    inputs are activation (real number between 0 and 1) and muscle fiber length lM
    outputs are muscle-tendon actuator force (fMT), d/dt of activation for next timestep (a_dot) and d/dt of fiber length (l_dot)

    Parameters
    
        VARYING PARAMETERS
        
        FM_o    (float):   maximum isometric force of muscle
        lM_o    (float):   optimal fiber length of muscle
        lT_s    (float):   tendon slack length (tendon length below which tendon produces no force)
        alpha_o (float):   pennation angle of the muscle fibers at optimal fiber length
        
        CONSTANT PARAMETERS
        eM_o    (float):   passive muscle strain due to maximum isometric force
        k_toe   (float):   exp shape factor
        eT_o    (float):   tendon strain due to maximum isometric force
        k_lin   (float):   linear shape factor
        eT_toe  (float):   tendon strain above which tendon force is linear w.r.t. tendon strain
        FT_toe  (float):   normalized tendon force above which tendon force is linear w.r.t. tendon strain
        kPE     (float):   exp shape factor for passive FL relationship
        gamma   (float):   shape factor for gaussian active FL relationship
        Af      (float):   FV shape factor 
        FM_len  (float):   max normalized muscle force when fiber is lengthening
        w       (float):   constant muscle width
        chi     (float):   passive damping factor
    """
    def __init__(self, activation, lM, lMT, FM_o = 3, lM_o = 0.3, lT_s = 0.5, alpha_o = 0):
        # muscle model inputs
        self.activation = activation
        self.lM = lM
        self.lMT = lMT
        
        # constants that may vary between muscle
        self.FM_o = FM_o
        self.lM_o = lM_o
        self.lT_s = lT_s
        self.alpha_o = alpha_o
        
        # constants that do not vary between muscle
        self.eM_o = 0.6
        self.k_toe = 3.0
        self.eT_o = 0.033
        self.k_lin = 1.712 / self.eT_o
        self.eT_toe = 0.609 * self.eT_o
        self.FT_toe = 1/3
        self.kPE = 4.0
        self.gamma = 0.5
        self.Af = 0.3
        self.FM_len = 1.8
        self.w = self.lM_o * np.sin(self.alpha_o)
        self.chi = 0.05
        self.e = 10**-6
        self.a = 0.5
    
    def _tendon_length(self,):
        ''' calculate tendon length '''
        alpha = self._pennation()
        return self.lMT - self.lM * np.cos(alpha)
        
    def _tendon_strain(self,):
        ''' calculate tendon strain '''
        norm_slack = self.lT_s / self.lM_o
        norm_tendon = self._tendon_length() / self.lM_o
        return (norm_tendon - norm_slack) / norm_slack
    
    def _tendon_force(self,):
        ''' calculate tendon force '''
        eT = self._tendon_strain()
        if eT > self.eT_toe:
            FT_bar = self.k_lin * (eT - self.eT_toe) + self.FT_toe
        elif 0 < eT and eT <= self.eT_toe:
            FT_bar = self.FT_toe * ( np.exp(self.k_toe * eT / self.eT_toe) - 1 ) / ( np.exp(self.k_toe) - 1 )
        else:
            FT_bar = 0
        return self.FM_o * 0.001 * ( 1 + eT ) * FT_bar
    
    def _pennation(self,):
        '''calculate pennation angle'''
        x = self.w / self.lM
        if self.lM == 0 or x <= 0:
            alpha = 0
        elif 0 < x and x < 1:
            alpha = np.arcsin(self.w / self.lM)
        elif 1 <= x:
            alpha = np.pi / 2
        return alpha
    
    def _active_force(self,):
        '''calculate active muscle force'''
        fl_bar = np.exp(- ( self.lM / self.lM_o - 1 )**2 / self.gamma )
        return self.activation * self.FM_o * fl_bar
    
    def _passive_force(self,):
        '''calculate active muscle force'''
        lM_bar = self.lM / self.lM_o
        if lM_bar > 1 + self.eM_o:
            FPE_bar = 1 + self.kPE / self.eM_o * ( lM_bar - ( 1 + self.eM_o ) )
        elif lM_bar <= (1 + self.eM_o):
            FPE_bar = np.exp( self.kPE * (lM_bar - 1) / self.eM_o ) / np.exp( self.kPE )
        return self.FM_o * FPE_bar
    
    def _contractile_force(self,):
        '''calculate contractile muscle force'''
        FT = self._tendon_force()
        alpha = self._pennation()
        FPE = self._passive_force()
        return FT / np.cos(alpha) - FPE
    
    def _inv_FV(self,):
        Fa = self._active_force()
        Fce = self._contractile_force()
        
        num = 0.95 * self.FM_len * Fa - Fa
        denom = ( (2 + 2/self.Af) * 0.05 * Fa * self.FM_len ) / (self.FM_len - 1) + self.chi
        f_vo = num / denom
        
        num1 = (0.95 + self.e) * self.FM_len * Fa - Fa
        denom1 = ( (2 + 2/self.Af) * ( 0.05 - self.e ) * Fa * self.FM_len ) / (self.FM_len - 1) + self.chi
        f_v1 = num1 / denom1
        
        if Fce < 0:
            fv_inv = Fce / self.e * ( (self.e - Fa) / (Fa + self.e / self.Af + self.chi) + Fa / (Fa + self.chi) ) - Fa / (Fa + self.chi)
        elif 0 <= Fce and Fce < Fa:
            fv_inv = (Fce - Fa) / (Fa + Fce / self.Af + self.chi)
        elif Fa <= Fce and Fce < 0.95*Fa*self.FM_len:
            fv_inv = (Fce - Fa) / ( ( (2 + 2/self.Af) * (Fa * self.FM_len - Fce) ) / (self.FM_len - 1) + self.chi )
        elif 0.95*Fa*self.FM_len <= Fce:
            fv_inv = f_vo + (Fce - 0.95*Fa*self.FM_len) / (self.e*Fa*self.FM_len) * (f_v1 - f_vo)
        
        return fv_inv
    
    def fiber_velocity(self,):
        l_dot_bar = self._inv_FV()
        Vmax = (5 + 5 * self.activation) * self.lM_o
        return l_dot_bar * Vmax
    
    def mt_force(self,):
        return self._tendon_force()